In [1]:
# Importing modules
import pandas as pd
import numpy as np
from config import config

# Reading the raw dataset
raw_dataset = pd.read_csv(config['path_raw'] + 'TACE_LR.csv')
print("The raw_dataset has the following shape: {}.".format(raw_dataset.shape))
raw_dataset.head()

The raw_dataset has the following shape: (5073, 22).


,ID,state,time,treatment_before,gender,age,Etiology,ECOG,tumor_number,tumor_size,...,TBIL,AST,ALT,PLT,INR,BUN,Cr,WBC,HGB,therapy
0,5002,1,26.618056,0,0,53.04783,1,0,2,18.0,...,22.9,161.000000,218.2,348.0,1.050000,5.556726,47.000000,8.07,61.2,TACE
1,5003,0,85.544444,0,1,59.76347,1,0,2,4.0,...,21.5,58.614578,23.4,37.0,1.155444,7.500000,87.000000,75.00,157.0,TACE
2,5004,0,1.666573,0,1,58.95445,1,0,2,9.8,...,28.1,95.000000,306.5,118.0,1.170000,4.600000,52.300000,62.00,131.0,TACE
3,5005,1,42.652778,0,1,68.12717,1,0,2,4.1,...,15.9,44.000000,13.8,115.0,1.260000,5.200000,73.088394,29.00,134.0,TACE
4,5006,0,12.486111,0,1,54.56005,1,0,2,4.1,...,34.5,24.000000,39.0,165.0,1.140000,5.556726,73.088394,20.90,148.5,TACE


In [2]:
# Defining the time and event column
time_column = 'time'
event_column = 'state'

# encode, drop...
import math
raw_dataset['time'] = raw_dataset['time'].apply(lambda x: math.ceil(x))
raw_dataset['therapy'] = raw_dataset['therapy'].apply(lambda x: 0 if x == "TACE" else 1)
raw_dataset['state'] = raw_dataset['state'].apply(lambda x: False if x == 0 else True)
dataset = raw_dataset.drop(['ID'], axis=1)
dataset_with_id = raw_dataset

# Defining the modeling features
features_all = np.setdiff1d(dataset.columns, ['time', 'state']).tolist()
features_select = ["tumor_size","therapy", "AFP", "AST", "tumor_number", "ECOG", "ALB", "HGB", "INR"]

# Checking for null values
N_null = sum(dataset[features_all].isnull().sum())
print("The dataset contains {} null values".format(N_null)) #0 null values

# Removing duplicates if there exist
N_dupli = sum(dataset.duplicated(keep='first'))
N_dupli_list = dataset.index[dataset.duplicated(keep='first') == True].tolist()
print(N_dupli_list)
dataset = dataset.drop_duplicates(keep='first').reset_index(drop=True)
dataset_with_id = dataset_with_id.drop(N_dupli_list, axis=0).reset_index(drop=True)
print("The dataset contains {} duplicates".format(N_dupli))

dataset_x = dataset_with_id[features_all]
dataset_y = dataset_with_id[["state", "time"]].to_records(index=False)

The dataset contains 0 null values
[316, 318, 339, 366, 389, 525, 605, 661, 664, 720, 723, 768, 780, 781, 786, 845, 852, 859, 864, 1215, 1298, 1302, 1304, 1313, 1317, 1396, 1401, 1408, 1410, 1498, 1501, 1503, 1505, 1511, 1608, 1624, 1628, 1742, 1749, 1894, 1897, 1901, 1904, 1908, 2019, 2022, 2157, 2167, 2178, 2184, 2350, 2352, 2483, 2502, 2513, 2518, 2882, 2891, 3106, 3114, 3355, 3358, 3362, 3378, 3577, 3741, 3751, 3760, 3765, 3917, 3922, 3932, 4069, 4089, 4264, 4266, 4407, 4412, 4420, 4721, 4805, 5042]
The dataset contains 82 duplicates


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
dataset_x, dataset_y, test_size=config['valid_ratio'], random_state=config['seed'])

In [4]:
print("The X_train has the following shape: {}.".format(X_train.shape))
print("The X_test has the following shape: {}.".format(X_test.shape))
print("The y_train has the following shape: {}.".format(y_train.shape))
print("The y_test has the following shape: {}.".format(y_test.shape))

The X_train has the following shape: (4192, 19).
The X_test has the following shape: (799, 19).
The y_train has the following shape: (4192,).
The y_test has the following shape: (799,).


In [11]:
from sksurv.ensemble import ComponentwiseGradientBoostingSurvivalAnalysis
cgb = ComponentwiseGradientBoostingSurvivalAnalysis(loss="coxph").fit(X_train, y_train)
print("The training set score is: {}".format(cgb.score(X_train, y_train)))
print("The test set score is: {}".format(cgb.score(X_test, y_test)))

The training set score is: 0.721144247296369
The test set score is: 0.7443302951105849


In [5]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
coxph = CoxPHSurvivalAnalysis().fit(X_train, y_train)
print("The training set score is: {}".format(coxph.score(X_train, y_train)))
print("The test set score is: {}".format(coxph.score(X_test, y_test)))

The training set score is: 0.7305692237713775
The test set score is: 0.7534549960120192


In [13]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
gb = GradientBoostingSurvivalAnalysis(loss="coxph").fit(X_train, y_train)
print("The training set score is: {}".format(gb.score(X_train, y_train)))
print("The test set score is: {}".format(gb.score(X_test, y_test)))

The training set score is: 0.7652051337806065
The test set score is: 0.7601618267460584


In [14]:
from sksurv.ensemble import ExtraSurvivalTrees
est = ExtraSurvivalTrees().fit(X_train, y_train)
print("The training set score is: {}".format(est.score(X_train, y_train)))
print("The test set score is: {}".format(est.score(X_test, y_test)))

The training set score is: 0.8224906079038695
The test set score is: 0.7525370079609127


In [15]:
from sksurv.ensemble import RandomSurvivalForest
estimator = RandomSurvivalForest().fit(dataset_x, dataset_y)
print("The training set score is: {}".format(estimator.score(X_train, y_train)))
print("The test set score is: {}".format(estimator.score(X_test, y_test)))

The training set score is: 0.9183335148504218
The test set score is: 0.919540604668195


In [32]:
# read the data
train = pd.read_csv(config["path_ok"] + "data_train.csv")
test = pd.read_csv(config["path_ok"] + "data_test.csv")

# encode, drop...
train['time'] = train['time'].apply(lambda x: math.ceil(x))
test['time'] = test['time'].apply(lambda x: math.ceil(x))
train['therapy'] = train['therapy'].apply(lambda x: 0 if x == 'TACE' else 1)
test['therapy'] = test['therapy'].apply(lambda x: 0 if x == 'TACE' else 1)
# train['therapy'] = train['therapy'].apply(lambda x: 1 if x == 'TACE' else 0)
# test['therapy'] = test['therapy'].apply(lambda x: 1 if x == 'TACE' else 0)

# Defining the modeling features
features_all = np.setdiff1d(train.columns, ['time', 'state', 'ID']).tolist()

X_train, X_test = train[features_all], test[features_all]
T_train, T_test = train[time_column], test[time_column]
E_train, E_test = train[event_column], test[event_column]

prs = estimator.predict_survival_function(X_train, True)
prs_test = estimator.predict_survival_function(X_test, True)

In [33]:
train = pd.read_csv(config["path_ok"] + "data_train.csv")
test = pd.read_csv(config["path_ok"] + "data_test.csv")

# concat prs
train_rsf = pd.concat([train, pd.DataFrame(prs)], axis=1)
test_rsf = pd.concat([test, pd.DataFrame(prs_test)], axis=1)

# concat time and event
from utils import *
v = 0.5

# 找到每一个样本的中位生存时间
time = []
for i in range(train.shape[0]):
    middle_day = prs[i].tolist().index(find_nearest(prs[i], v))
    time.append(middle_day)
train_rsf['pred_time'] = time
train_rsf['pred_state'] = 1

time = []
for i in range(test.shape[0]):
    middle_day = prs_test[i].tolist().index(find_nearest(prs_test[i], v))
    time.append(middle_day)
test_rsf['pred_time'] = time
test_rsf['pred_state'] = 1

In [34]:
# save the result
train_rsf.to_csv('./data/'+ "RSF_train_before.csv", index=False)
test_rsf.to_csv('./data/' + "RSF_test_before.csv", index=False)
# train_rsf.to_csv('./data/'+ "RSF_train_after.csv", index=False)
# test_rsf.to_csv('./data/' + "RSF_test_after.csv", index=False)

In [7]:
import numpy as np

def fit_and_score_features(X, y):
    n_features = X.shape[1]
    scores = np.empty(n_features)
    m = coxph
    for j in range(n_features):
        Xj = X[:, j:j+1]
        m.fit(Xj, y)
        scores[j] = m.score(Xj, y)
    return scores

sc = fit_and_score_features(dataset_x.values, dataset_y)
pd.Series(sc, index=dataset_x.columns).sort_values(ascending=False)

tumor_size          0.669369
therapy             0.633784
AFP                 0.610519
AST                 0.599039
tumor_number        0.594154
ECOG                0.573752
ALB                 0.570808
HGB                 0.560430
INR                 0.555996
WBC                 0.542874
BUN                 0.537982
ALT                 0.530191
TBIL                0.514794
treatment_before    0.511717
Cr                  0.509072
Etiology            0.508923
PLT                 0.506582
gender              0.505376
age                 0.500892
dtype: float64

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sksurv.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, KFold

pipe = Pipeline([('encode', OneHotEncoder()),
                 ('select', SelectKBest(fit_and_score_features, k=3)),
                 ('model', coxph)])
param_grid = {'select__k': np.arange(1, dataset_x.shape[1] + 1)}
cv = KFold(n_splits=3, random_state=42, shuffle=True)
gcv = GridSearchCV(pipe, param_grid, return_train_score=True, cv=cv)
gcv.fit(dataset_x, dataset_y)

results = pd.DataFrame(gcv.cv_results_).sort_values(by='mean_test_score', ascending=False)
results.loc[:, ~results.columns.str.endswith("_time")]